In [2]:
%pip install -r requirements.txt

  Cloning https://github.com/openai/CLIP.git (to revision a1d071733d7111c9c014f024669f959182114e33) to /private/var/folders/4_/051f5l115s33r4zvqhcsst0r0000gn/T/pip-install-_8jc7wb7/clip_31378a0dad594d179af4de51c841b944
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/4_/051f5l115s33r4zvqhcsst0r0000gn/T/pip-install-_8jc7wb7/clip_31378a0dad594d179af4de51c841b944
  Running command git rev-parse -q --verify 'sha^a1d071733d7111c9c014f024669f959182114e33'
  Running command git fetch -q https://github.com/openai/CLIP.git a1d071733d7111c9c014f024669f959182114e33
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the 

In [34]:
import pandas 

df = pandas.read_csv('./mincult-train/train.csv', sep=';')
df = df[:30]
df['path'] = './mincult-train/train/'+ df['object_id'].astype(str) + '/' + df['img_name']
df['embedding_path'] = df['path']+'.embedding'

In [26]:
df.head()

,object_id,name,description,group,img_name,./mincult-train/train/4782780/31762267.jpg
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg,./mincult-train/train/10669820/7862029.jpg
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg,./mincult-train/train/4489444/9461061.jpg
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg,./mincult-train/train/8722586/5095122.jpg
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg,./mincult-train/train/3712248/551422.jpg
4,6339754,Сабля.,NaN,Оружие,2592073.jpg,./mincult-train/train/6339754/2592073.jpg


In [12]:
from PIL import Image
import clip
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device}")
device = torch.device(device)
model, preprocess = clip.load("ViT-L/14")
model.to(device)

using cpu


In [27]:
def get_embedding(image_path): 
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        return model.encode_image(image)[0].tolist()

In [36]:
df['embedding'] = df['path'].map()
for index, row in df.iterrows(): 
    embedding = get_embedding(row.path)
    with open(row.embedding_path, 'w') as f:
        f.write(str(embedding))

In [37]:
df.head()

,object_id,name,description,group,img_name,path,embedding_path
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg,./mincult-train/train/10669820/7862029.jpg,./mincult-train/train/10669820/7862029.jpg.emb...
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg,./mincult-train/train/4489444/9461061.jpg,./mincult-train/train/4489444/9461061.jpg.embe...
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg,./mincult-train/train/8722586/5095122.jpg,./mincult-train/train/8722586/5095122.jpg.embe...
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg,./mincult-train/train/3712248/551422.jpg,./mincult-train/train/3712248/551422.jpg.embed...
4,6339754,Сабля.,NaN,Оружие,2592073.jpg,./mincult-train/train/6339754/2592073.jpg,./mincult-train/train/6339754/2592073.jpg.embe...


In [80]:
%pip install scipy
from scipy.spatial import distance

def find_best(image_path): 
    emb = get_embedding(image_path) 
    distances = {} 
    for index, row in df.iterrows(): 
        with open(row.embedding_path, 'r') as f:
            other_emb = eval(f.readline()) # TODO: parse correctly
            dist = distance.cosine(emb, other_emb)
            distances[index] = dist
    dists = sorted(list(distances.items()), key=lambda a: a[1])[:10] 
    for i, dist in dists: 
        print(df.loc[[i]].path)


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [83]:
find_best('swordjpg')

0.18709602847776197 4    ./mincult-train/train/6339754/2592073.jpg
Name: path, dtype: object
0.2142694655474221 1    ./mincult-train/train/4489444/9461061.jpg
Name: path, dtype: object
0.22828367408897454 27    ./mincult-train/train/4780854/1267334.jpg
Name: path, dtype: object
0.2572552842243817 2    ./mincult-train/train/8722586/5095122.jpg
Name: path, dtype: object
0.2614007460044081 21    ./mincult-train/train/6339644/2592023.jpg
Name: path, dtype: object
0.264012111849101 19    ./mincult-train/train/21615364/22970356.jpg
Name: path, dtype: object
0.27966460032043894 12    ./mincult-train/train/9871461/6667278.jpg
Name: path, dtype: object
0.2798171518531123 16    ./mincult-train/train/24124732/26459574.jpg
Name: path, dtype: object
0.28023764331738477 6    ./mincult-train/train/4704612/2008677.jpg
Name: path, dtype: object
0.319001116746622 23    ./mincult-train/train/46777447/57808991.jpg
Name: path, dtype: object
